<a href="https://colab.research.google.com/github/chrisdemario/Evolve/blob/main/EvolveDensityPatches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Colab-ready script to generate three PNG layers, convert them to 1-bit, rename as .prt, and zip
# Run this directly in Google Colab

from PIL import Image, ImageDraw
import os, zipfile
from google.colab import files

# DPI and page size
dpi = 600
page_width_px, page_height_px = 7200, 7200  # square image size

# Patch size and spacing
patch_size_in = 1.245
patch_size_px = int(patch_size_in * dpi)
spacing_in = 1.5
spacing_px = int(spacing_in * dpi)

# Repeat spacing along length
repeat_in = 2.0
repeat_px = int(repeat_in * dpi)

# Output directory (safe for Colab)
output_dir = "/content/DensityPatches"
os.makedirs(output_dir, exist_ok=True)

# Function to draw layer
def draw_layer(active_columns, filename):
    img = Image.new("L", (page_width_px, page_height_px), color=0)
    draw = ImageDraw.Draw(img)

    # Center grid horizontally
    total_width = (len(active_columns) - 1) * spacing_px + patch_size_px
    x_start = (page_width_px - total_width) // 2

    # Compute total vertical height of repeated patches
    num_repeats = page_height_px // repeat_px
    total_height = (num_repeats - 1) * repeat_px + patch_size_px
    y_start = (page_height_px - total_height) // 2

    for i, active in enumerate(active_columns):
        if active:
            x = x_start + i * spacing_px
            for j in range(num_repeats):
                y = y_start + j * repeat_px
                draw.rectangle([x, y, x + patch_size_px, y + patch_size_px], fill=255)

    out_file = os.path.join(output_dir, filename)
    img.save(out_file, dpi=(dpi, dpi))
    print(f"Saved {out_file}")
    return out_file

# Define which columns are active for each image
layer_1_cols = [0,1,1,1,1,0,0]
layer_2_cols = [1,1,0,1,0,0,1]
layer_3_cols = [0,0,0,1,1,1,1]

# Draw layers
paths = []
paths.append(draw_layer(layer_1_cols, "layer_1.png"))
paths.append(draw_layer(layer_2_cols, "layer_2.png"))
paths.append(draw_layer(layer_3_cols, "layer_3.png"))

# Convert PNGs to 1-bit, save as PNGs, then rename to .prt
prt_files = []
for i, p in enumerate(paths, start=1):
    img = Image.open(p).convert("1")  # 1-bit per pixel
    tmp_png = os.path.join(output_dir, f"{i}.png")
    img.save(tmp_png)
    prt_name = os.path.join(output_dir, f"{i}.prt")
    os.replace(tmp_png, prt_name)  # atomic rename if possible
    print(f"Saved {prt_name}")
    prt_files.append(prt_name)

# Simple verification (opens by content signature, not extension)
for f in prt_files:
    try:
        with Image.open(f) as im:
            im.load()
            assert im.mode in ("1", "L"), f"Unexpected mode: {im.mode}"
    except Exception as e:
        print(f"WARNING: Could not verify {f}: {e}")

# Zip the .prt files
zip_path = "/content/DensityPatches.zip"
with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    for f in prt_files:
        zipf.write(f, os.path.basename(f))

print(f"Created zip archive at {zip_path}")

# Offer zip file for download
files.download(zip_path)


Saved /content/DensityPatches/layer_1.png
Saved /content/DensityPatches/layer_2.png
Saved /content/DensityPatches/layer_3.png
Saved /content/DensityPatches/1.prt
Saved /content/DensityPatches/2.prt
Saved /content/DensityPatches/3.prt
Created zip archive at /content/DensityPatches.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>